In [9]:
style = """
<style>
.reveal pre.xr-text-repr-fallback {
    display: none;
}


.reveal ul.xr-sections {
    padding-left: 0 !important;
    display: grid;
    grid-template-columns: 150px auto auto 1fr 20px 20px;
}

.reveal .xr-wrap {
    font-size: 70%;
    max-width: 1000px;
}

.reveal .xr-var-data table {
    font-size: 90%;
    
}

.reveal .xr-var-list, .xr-var-item {
    display: contents;
}
</style>
"""

from IPython.core.display import HTML
HTML(style)

# Pangeo ❤️ OSN

## Interactive Data-Proximate Computing in the Cloud 

# Xarray + Zarr

In [34]:
import xarray as xr
url = ("https://ncsa.osn.xsede.org/Pangeo/pangeo-forge/swot_adac/"
       "eNATL60/Region01/surface_hourly/fma.zarr")
ds = xr.open_zarr(url, consolidated=True)
ds

<xarray.Dataset>
Dimensions:        (time_counter: 2136, x: 611, y: 763)
Coordinates: (12/15)
    depth          (y, x) float32 dask.array<chunksize=(763, 611), meta=np.ndarray>
    e1f            (y, x) float64 dask.array<chunksize=(763, 611), meta=np.ndarray>
    e1t            (y, x) float64 dask.array<chunksize=(763, 611), meta=np.ndarray>
    e1u            (y, x) float64 dask.array<chunksize=(763, 611), meta=np.ndarray>
    e1v            (y, x) float64 dask.array<chunksize=(763, 611), meta=np.ndarray>
    e2f            (y, x) float64 dask.array<chunksize=(763, 611), meta=np.ndarray>
    ...             ...
    lat            (y, x) float32 dask.array<chunksize=(763, 611), meta=np.ndarray>
    lon            (y, x) float32 dask.array<chunksize=(763, 611), meta=np.ndarray>
    nav_lat        (y, x) float32 dask.array<chunksize=(763, 611), meta=np.ndarray>
    nav_lon        (y, x) float32 dask.array<chunksize=(763, 611), meta=np.ndarray>
    time_centered  (time_counter) datetime64[ns] dask.array<chunksize=(72,), meta=np.ndarray>
  * time_counter   (time_counter) datetime64[ns] 2010-02-01T00:30:00 ... 2010...
Dimensions without coordinates: x, y
Data variables: (12/13)
    fmask          (y, x) int8 dask.array<chunksize=(763, 611), meta=np.ndarray>
    qt_oce         (time_counter, y, x) float32 dask.array<chunksize=(72, 763, 611), meta=np.ndarray>
    somecrty       (time_counter, y, x) float32 dask.array<chunksize=(72, 763, 611), meta=np.ndarray>
    sometauy       (time_counter, y, x) float32 dask.array<chunksize=(72, 763, 611), meta=np.ndarray>
    sosaline       (time_counter, y, x) float32 dask.array<chunksize=(72, 763, 611), meta=np.ndarray>
    sossheig       (time_counter, y, x) float32 dask.array<chunksize=(72, 763, 611), meta=np.ndarray>
    ...             ...
    sowaflup       (time_counter, y, x) float32 dask.array<chunksize=(72, 763, 611), meta=np.ndarray>
    sozocrtx       (time_counter, y, x) float32 dask.array<chunksize=(72, 763, 611), meta=np.ndarray>
    sozotaux       (time_counter, y, x) float32 dask.array<chunksize=(72, 763, 611), meta=np.ndarray>
    tmask          (y, x) int8 dask.array<chunksize=(763, 611), meta=np.ndarray>
    umask          (y, x) int8 dask.array<chunksize=(763, 611), meta=np.ndarray>
    vmask          (y, x) int8 dask.array<chunksize=(763, 611), meta=np.ndarray>

In [14]:
import numpy as np
ds = ds.assign_coords({'x': ('x', np.arange(len(ds.x))),
                       'y': ('y', np.arange(len(ds.y)))})

In [24]:
del ds['x']
del ds['y']

In [25]:
import hvplot.xarray
ds.sosstsst.hvplot.image('x', 'y',
                         rasterize=True, dynamic=True, width=800, height=450,
                         widget_type='scrubber', widget_location='bottom', cmap='magma')

DataError: Supplied data does not contain specified dimensions, the following dimensions were not found: ['x', 'y']

Interface expects tabular data, for more information on supported datatypes see http://holoviews.org/user_guide/Tabular_Datasets.html

In [32]:
da = xr.DataArray(np.ones((2, 10, 10)), dims=['time', 'y', 'x'],
                  coords={'time': ('time', [0, 1])})
da.hvplot.image('x', 'y')

ValueError: cannot select an axis to squeeze out which has size not equal to one

:Image   [x,y]   (value)

In [18]:
da

<xarray.DataArray (x: 10, y: 10)>
array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])
Dimensions without coordinates: x, y